## 1단계: 필수 패키지 설치

In [1]:
!pip install --upgrade azure-ai-evaluation
!pip install pandas numpy matplotlib seaborn

## 2단계: 라이브러리 임포트 및 데이터 로드

In [2]:
import os
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# Microsoft Foundry SDK - QA Evaluator
try:
    from azure.ai.evaluation import QAEvaluator
    print("✓ Azure AI Evaluation 임포트 성공")
except ImportError:
    print("⚠ Azure AI Evaluation 설치 필요: pip install azure-ai-evaluation")

# 데이터 로드
with open('data/sample_qa.json', 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

df = pd.DataFrame(qa_data)

print(f"\n✓ 데이터 로드 완료: {len(df)} 개 Q&A 쌍")
print(f"\n에이전트별 분포:")
print(df['agent'].value_counts())

✓ Azure AI Evaluation 임포트 성공

✓ 데이터 로드 완료: 34 개 Q&A 쌍

에이전트별 분포:
agent
product-agent          10
hallucination-agent     7
review-agent            6
policy-agent            6
rag-fail-agent          5
Name: count, dtype: int64


## 3단계: QA Evaluator 초기화

In [ ]:
# ============================================================
# model_config 설정 (직접 입력 값이 있으면 우선 사용)
# ============================================================
model_config = {
    "azure_endpoint": "https://your-endpoint.openai.azure.com/",
    "api_key": "your-api-key",
    "azure_deployment": "gpt-4",
    "api_version": "2024-12-01-preview"
}

# QA Evaluator 초기화 (model_config 필수)
try:
    qa_evaluator = QAEvaluator(model_config=model_config)
    
    print("\n✓ QAEvaluator 초기화 성공")
    print("\n📋 QAEvaluator 메트릭:")
    print("  - groundedness: 근거성 (1-5)")
    print("  - relevance: 관련성 (1-5)")
    print("  - coherence: 일관성 (1-5)")
    print("  - fluency: 유창성 (1-5)")
except Exception as e:
    print(f"\n⚠ Evaluator 초기화 실패")
    print(f"에러: {str(e)[:300]}")
    print("\n🔧 해결 방법:")
    print("1. 위 셀에서 AZURE_OPENAI_* 변수에 직접 값 입력")
    print("2. 또는 .env 파일 생성 후 커널 재시작")


✓ QAEvaluator 초기화 성공

📋 QAEvaluator 메트릭:
  - groundedness: 근거성 (1-5)
  - relevance: 관련성 (1-5)
  - coherence: 일관성 (1-5)
  - fluency: 유창성 (1-5)


## 4단계: QA Evaluator 실행

In [4]:

qa_results = []

def decode_nested_json_unicode(obj, max_depth=5):
    """다중 중첩된 JSON 문자열의 유니코드 이스케이프를 재귀적으로 디코딩"""
    if isinstance(obj, str):
        # JSON 문자열 여부 확인 및 반복 파싱
        current = obj
        for _ in range(max_depth):
            if current.startswith(('[', '{')):
                try:
                    parsed = json.loads(current)
                    # 파싱된 내용을 재귀 처리
                    decoded = decode_nested_json_unicode(parsed, max_depth - 1)
                    # 다시 JSON 문자열로 변환 (ensure_ascii=False로 한글 유지)
                    current = json.dumps(decoded, ensure_ascii=False)
                except:
                    break
            else:
                break
        return current
    elif isinstance(obj, dict):
        return {k: decode_nested_json_unicode(v, max_depth) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [decode_nested_json_unicode(item, max_depth) for item in obj]
    else:
        return obj

for idx, row in df.iterrows():
    try:
        # QAEvaluator 실행
        result = qa_evaluator(
            query=row['question'],
            response=row['response'],
            context=row.get('context', row['question']),
            ground_truth=row.get('ground_truth', '')
        )
        
        # 다중 중첩 JSON의 유니코드 이스케이프 디코딩
        decoded_result = {}
        for key, value in result.items():
            decoded_result[key] = decode_nested_json_unicode(value)
        
        decoded_result['agent'] = row['agent']
        qa_results.append(decoded_result)
        
    except Exception as e:
        print(f"✗ 오류: {str(e)[:80]}")

# 결과 표시
if qa_results:
    qa_df = pd.DataFrame(qa_results)
    
    # pandas 출력 옵션
    pd.set_option('display.max_colwidth', 100)
    pd.set_option('display.max_columns', None)
    
    print(f"\n=== 샘플 평가 결과 ({len(qa_results)}개) ===\n")
    
    try:
        from IPython.display import display
        display(qa_df)
    except:
        print(qa_df.to_string(index=False))
else:
    print("\n평가 결과가 없습니다.")


=== 샘플 평가 결과 (34개) ===



,f1_score,f1_result,f1_threshold,similarity,gpt_similarity,similarity_result,similarity_threshold,similarity_prompt_tokens,similarity_completion_tokens,similarity_total_tokens,similarity_finish_reason,similarity_model,similarity_sample_input,similarity_sample_output,relevance,gpt_relevance,relevance_result,relevance_threshold,relevance_reason,relevance_prompt_tokens,relevance_completion_tokens,relevance_total_tokens,relevance_finish_reason,relevance_model,relevance_sample_input,relevance_sample_output,groundedness,gpt_groundedness,groundedness_reason,groundedness_result,groundedness_threshold,groundedness_prompt_tokens,groundedness_completion_tokens,groundedness_total_tokens,groundedness_finish_reason,groundedness_model,groundedness_sample_input,groundedness_sample_output,fluency,gpt_fluency,fluency_reason,fluency_result,fluency_threshold,fluency_prompt_tokens,fluency_completion_tokens,fluency_total_tokens,fluency_finish_reason,fluency_model,fluency_sample_input,fluency_sample_output,coherence,gpt_coherence,coherence_reason,coherence_result,coherence_threshold,coherence_prompt_tokens,coherence_completion_tokens,coherence_total_tokens,coherence_finish_reason,coherence_model,coherence_sample_input,coherence_sample_output,agent
0,0.000000,pass,3,5.0,5.0,pass,3,891,1,892,length,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""갤럭시 S24의 배터리 용량이 얼마나 되나요?\"", \""response\"": \""갤럭시 S24...","[{""role"": ""assistant"", ""content"": ""5""}]",4.0,4.0,pass,3,The response directly answers the user's question by providing the exact battery capacity of the...,1639,53,1692,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""갤럭시 S24의 배터리 용량이 얼마나 되나요?\"", \""response\"": \""갤럭시 S24...","[{""role"": ""assistant"", ""content"": ""{\""explanation\"": \""The response directly answers the user's ...",5.0,5.0,The response correctly and fully answers the question about battery capacity using the exact fig...,pass,3,1428,205,1633,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""갤럭시 S24의 배터리 용량이 얼마나 되나요?\"", \""response\"": \""갤럭시 S24...","[{""role"": ""assistant"", ""content"": ""<S0>Let's think step by step: The query asks specifically abo...",3.0,3.0,"The response is clear, coherent, and grammatically correct with adequate vocabulary and sentence...",pass,3,962,150,1112,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""response\"": \""갤럭시 S24의 배터리 용량은 4,000mAh입니다. 25W 고속 충전과 15W 무선 충...","[{""role"": ""assistant"", ""content"": ""<S0>Let's think step by step: The response is written in Kore...",5.0,5.0,"The response is coherent because it directly answers the question with clear, logically connecte...",pass,3,1317,149,1466,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""갤럭시 S24의 배터리 용량이 얼마나 되나요?\"", \""response\"": \""갤럭시 S24...","[{""role"": ""assistant"", ""content"": ""<S0>Let's think step by step: The query asks specifically abo...",product-agent
1,0.296296,pass,3,5.0,5.0,pass,3,902,1,903,length,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""노트북 A1 모델의 RAM은 업그레이드 가능한가요?\"", \""response\"": \""노트북 ...","[{""role"": ""assistant"", ""content"": ""5""}]",5.0,5.0,pass,3,The response directly answers the user's question about whether the RAM in the A1 laptop model i...,1641,62,1703,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""노트북 A1 모델의 RAM은 업그레이드 가능한가요?\"", \""response\"": \""노트북 ...","[{""role"": ""assistant"", ""content"": ""{\""explanation\"": \""The response directly answers the user's ...",5.0,5.0,"The response is fully correct and complete, directly answering the question with all relevant de...",pass,3,1427,165,1592,stop,gpt-4.1-mini-2025-04-14,"[{""role"": ""user"", ""content"": ""{\""query\"": \""노트북 A1 모델의 RAM은 업그레이드 가능한가요?\"", \""response\"": \""노트북 ...","[{""role"": ""assistant"", ""content"": ""<S0>Let's think step by 

## 5단계: 멀티턴 대화 평가 (Coherence, Fluency, F1Score)

멀티턴 대화(5개 대화 = user/assistant 교차 10개 메시지)를 CoherenceEvaluator, FluencyEvaluator, F1ScoreEvaluator로 평가합니다. 이들 evaluator는 `conversation` 파라미터를 지원하여 전체 대화 흐름을 평가합니다.

### Evaluator 특징 및 활용 시나리오

| Evaluator | 평가 대상 | 점수 범위 | 멀티턴 지원 | 활용 시나리오 |
|-----------|---------|---------|----------|-----------|
| **CoherenceEvaluator** | 대화 흐름의 논리적 일관성 | 1-5 | ✅ | 챗봇 대화의 맥락 유지도, 질문-응답 연결성 평가 |
| **FluencyEvaluator** | 응답의 자연스러움 및 문법 | 1-5 | ✅ | 챗봇 응답 품질, 언어 자연성, 이해하기 쉬운 정도 |
| **F1ScoreEvaluator** | 응답과 정답의 정확도 (토큰 기반) | 0-1 | ✅ | 사실 기반 QA, 정보 추출 정확도, 답변 정확성 검증 |

#### 각 Evaluator 활용 예시

**CoherenceEvaluator**
- 사용 사례: 고객 지원 챗봇, 교육 튜터 AI, 장시간 대화 세션
- 목표: 대화 중 주제가 일관되는지, 이전 대화를 기반으로 응답하는지 검증

**FluencyEvaluator**
- 사용 사례: 챗봇 응답 품질 관리, 번역 품질 평가, 자동 텍스트 생성
- 목표: 응답이 자연스럽고 읽기 쉬운지, 문법적으로 올바른지 평가

**F1ScoreEvaluator**
- 사용 사례: 정보 검색 QA, FAQ 챗봇, 사실 기반 답변 시스템
- 목표: 응답과 예상 정답의 토큰 기반 일치도를 정량적으로 측정 (멀티턴 대화의 전체 또는 마지막 응답 평가 가능)

In [5]:
from azure.ai.evaluation import CoherenceEvaluator, FluencyEvaluator, F1ScoreEvaluator

# 멀티턴 샘플 대화 로드 (JSON 파일에서)
with open('data/multi_turn_conversations_samples.json', 'r', encoding='utf-8') as f:
    sample_conversations = json.load(f)

# 첫 번째와 두 번째 샘플 사용 (정확한 대화 vs 불일치 대화)
good_sample = sample_conversations[0]
bad_sample = sample_conversations[1]

conversation_good = {"messages": good_sample["messages"]}
conversation_bad = {"messages": bad_sample["messages"]}

# Evaluators 초기화
coherence_eval = CoherenceEvaluator(model_config=model_config)
fluency_eval = FluencyEvaluator(model_config=model_config)
f1_eval = F1ScoreEvaluator()

print("=== 정확한 대화 vs 불일치한 대화 비교 ===\n")
print(f"데이터 출처: data/multi_turn_conversations_samples.json\n")

# 평가 실행
results_list = []

for label, conversation in [(good_sample["label"], conversation_good), (bad_sample["label"], conversation_bad)]:
    results = {"대화 유형": label, "대화수": len(conversation["messages"])//2}
    
    #print(conversation)

    try:
        r = coherence_eval(conversation=conversation)
        for k, v in r.items():
            if isinstance(v, (int, float)):
                results[f"coherence_{k}"] = round(v, 2)
    except Exception as e:
        results["coherence_error"] = "오류"
    
    try:
        r = fluency_eval(conversation=conversation)
        for k, v in r.items():
            if isinstance(v, (int, float)):
                results[f"fluency_{k}"] = round(v, 2)
    except Exception as e:
        results["fluency_error"] = "오류"
    
    try:
        # F1Score도 전체 대화 기반으로 평가
        r = f1_eval(conversation=conversation)
        for k, v in r.items():
            if isinstance(v, (int, float)):
                results[f"f1_{k}"] = round(v, 2)
    except Exception as e:
        results["f1_error"] = "오류"
    
    results_list.append(results)

# 결과 비교 표
from IPython.display import display
comparison_df = pd.DataFrame(results_list)
display(comparison_df)

print("\n📊 해석:")
print("- Coherence: 높을수록 대화 흐름이 일관성 있고 논리적")
print("- Fluency: 높을수록 응답이 자연스럽고 문법적으로 올바름")
print("- F1 Score: 높을수록 예상 정답과 일치도가 높음")

=== 정확한 대화 vs 불일치한 대화 비교 ===

데이터 출처: data/multi_turn_conversations_samples.json



,대화 유형,대화수,coherence_coherence,coherence_gpt_coherence,coherence_coherence_threshold,coherence_coherence_prompt_tokens,coherence_coherence_completion_tokens,coherence_coherence_total_tokens,fluency_fluency,fluency_gpt_fluency,fluency_fluency_threshold,fluency_fluency_prompt_tokens,fluency_fluency_completion_tokens,fluency_fluency_total_tokens,f1_f1_score,f1_f1_threshold
0,✅ 정확한 대화,5,4.0,4.0,3.0,1292.4,160.8,1453.2,3.2,3.2,3.0,944.4,151.6,1096.0,0.0,0.5
1,❌ 불일치 대화,5,1.6,1.6,3.0,1279.6,169.2,1448.8,2.6,2.6,3.0,931.6,193.6,1125.2,0.0,0.5



📊 해석:
- Coherence: 높을수록 대화 흐름이 일관성 있고 논리적
- Fluency: 높을수록 응답이 자연스럽고 문법적으로 올바름
- F1 Score: 높을수록 예상 정답과 일치도가 높음
